In [ ]:
"""
Optuna example that optimizes multi-layer perceptrons using PyTorch.
In this example, we optimize the validation accuracy of hand-written digit recognition using
PyTorch and MNIST. We optimize the neural network architecture as well as the optimizer
configuration. As it is too time consuming to use the whole MNIST dataset, we here use a small
subset of it.
"""

import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

import time

import pandas as pd 

from full_data_load_ep import *
from data_processing import *

import optuna


DEVICE = torch.device("cpu")
DIR = os.getcwd()
EPOCHS = 10
LOG_INTERVAL = 10


def define_model(trial):
    # We optimize the number of layers, hidden untis and dropout ratio in each layer.
    lstm_layers = trial.suggest_int("lstm_layers", 2, 4)
    lstm_hidden = trial.suggest_int("lstm_hidden", 74, 74 * 3)

    return Model(lstm_layers=lstm_layers, hidden_size=lstm_hidden)

class Model(nn.Module):
    def __init__(self, input_size=74, hidden_size=74, lstm_layers=3, output_size=1, drop=0.2):
        
        super().__init__()
        self.start = time.time()
        self.hidden_size = hidden_size
        self.lstm_layers = lstm_layers
        self.lstm = nn.LSTM(input_size, hidden_size, lstm_layers, batch_first=True,
#                             dropout=drop
                           )
        self.linear = nn.Linear(hidden_size, output_size)  
        self.relu = nn.ReLU()

    def forward(self, seasons):

        ht = torch.zeros(self.lstm_layers, 1, self.hidden_size)   # initialize hidden state
        ct = torch.zeros(self.lstm_layers, 1, self.hidden_size)  # initialize cell state
        predictions = torch.Tensor([]) # to store our predictions for season t+1
        
        hidden = (ht, ct)
        
        for idx, season in enumerate(seasons):  # here we want to iterate over the time dimension
            lstm_input = torch.FloatTensor(season).view(1,1,len(season)) # LSTM takes 3D tensor
            out, hidden = self.lstm(lstm_input, hidden) # LSTM updates hidden state and returns output
            pred_t = self.linear(out) # pass LSTM output through a linear activation function
            pred_t = self.relu(pred_t) # since performance is non-negative we apply ReLU
            
            predictions = torch.cat((predictions, pred_t)) # concatenate all the predictions

        return predictions

def get_player_dataset(target='ppg_y_plus_1'):

    df = pd.read_csv('../data/player_season_stats.csv')

    X, y,_ = prepare_features(df, target)

    players = X.index.droplevel(-1).unique() # get number indices
    n_players = players.shape[0] # get number of players
    train_idx, test_idx = train_test_split(players, test_size=0.3, random_state=18)

    print('--- Padding Player Data ---')

    X = pad_data(X.reset_index(), players)
    y = pad_data(y.reset_index(), players)

    X.set_index(['playerid', 'player', 'season_age'], inplace=True)
    y.set_index(['playerid', 'player', 'season_age'], inplace=True)

    print('--- Generating Player Data ---')
    train_seq, train_target = generate_players(X, y, train_idx)
    test_seq, test_target = generate_players(X, y, test_idx)

    train_idx_bool = pd.Series(list(X.index.droplevel(-1))).isin(train_idx).values
    test_idx_bool = pd.Series(list(X.index.droplevel(-1))).isin(test_idx).values

    train_real_values = (X[train_idx_bool] != -1).all(axis=1)
    test_real_values = (X[test_idx_bool] != -1).all(axis=1)

    return X, y, train_seq, train_target, test_seq, test_target, train_idx, test_idx, train_real_values, test_real_values, train_idx_bool, test_idx_bool

def objective(trial):

    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-3, 1e-1, log=True)
    print('Optimizer: ', optimizer_name, ' LR: ',lr)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    loss_fn = nn.MSELoss()

    # Get the player dataset.
    X, y, train_seq, train_target, test_seq, test_target, train_idx, test_idx, train_real_values, test_real_values, train_idx_bool, test_idx_bool = get_player_dataset()

    # to track the training loss as the model trains
    train_losses = []
    # to track the validation loss as the model trains
    valid_losses = []
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = [] 

    # Training of the model.
    for epoch in range(EPOCHS):
#         model.train()
        for seasons, targets in zip(train_seq, train_target):

            optimizer.zero_grad()
            mask = torch.tensor([(season > -1).all() for season in seasons]) # create mask for real seasons only      
            targets = torch.FloatTensor(targets)[mask]
            
            predictions = model(seasons)
            
            loss = loss_fn(predictions[mask].squeeze(1), targets) 
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        # Validation of the model.
#         model.eval()
        val_loss = []
        with torch.no_grad():
            for seasons, targets in zip(test_seq, test_target):
                # Limiting validation data.

                mask = torch.tensor([(season > -1).all() for season in seasons]) # create mask for real seasons only      
                targets = torch.FloatTensor(targets)[mask]
                
                predictions = model(seasons)
                loss = loss_fn(predictions[mask].squeeze(1), targets)
                valid_losses.append(loss.item())

        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)

        print_msg = (f'[{epoch:>{epoch}}/{EPOCHS:>{epoch}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')
        
        print(print_msg)
        
        # clear lists to track next epoch
        train_losses = []
        valid_losses = []

        trial.report(train_loss, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return train_loss

def train_model(trial):

    # Generate the model.
#     model = define_model(trial).to(DEVICE)
    model = Model(lstm_layers=4, hidden_size=74*3)

    # Generate the optimizers.
    lr = trial.suggest_float("lr", 1e-3, 1e-1, log=True)

    # to track the training loss as the model trains
    train_losses = []
    # to track the validation loss as the model trains
    valid_losses = []
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = [] 

    # old trainer
    trainer = Trainer(train_seq, train_target, test_seq, test_target, model, epochs=EPOCHS, lr=lr) # best run: {'epochs' : 100, 'lr' : 0.01}
    trainer.train(trial)

    return np.nanmean(trainer.train_loss)

class Trainer(object):
    def __init__(self, train_sequences, train_targets, test_sequences, test_targets, model, epochs=200, lr=0.01, batch_size=1, log_per=10000):
        self.start = time.time()
        self.epochs = epochs
        self.log_per = log_per
        self.train_sequences = train_sequences
        self.train_targets = train_targets
        self.test_sequences = test_sequences
        self.test_targets = test_targets
        self.batch_size = batch_size
        self.lr = lr
        self.loss_fn = nn.MSELoss()
        self.model = model
        
    def train(self, trial):

        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        
        train_epoch_loss = []
        test_epoch_loss = []
        
        for ep in range(1, self.epochs):
            train_loss = []
            test_loss = []
            print(f'Running epoch: {ep}')
            for seasons, targets in zip(self.train_sequences, self.train_targets): # data is a list returning tuple of X, y

                optimizer.zero_grad()
                        
                mask = torch.tensor([(season > -1).all() for season in seasons]) # create mask for real seasons only      
                targets = torch.FloatTensor(targets)[mask]
                
                predictions = self.model(seasons)    
                # now here, we want to compute the loss between the predicted values
                # for each season and the actual values for each season
                # TO-DO: random select grouth truth or predicted value in next timestep
                loss = self.loss_fn(predictions[mask].squeeze(1), targets) 
                loss.backward()
                optimizer.step() 
                train_loss.append(loss.item())
            
            # validate with test set
            with torch.no_grad():
                for seasons, targets in zip(self.test_sequences, self.test_targets):
                    mask = torch.tensor([(season > -1).all() for season in seasons]) # create mask for real seasons only      
                    targets = torch.FloatTensor(targets)[mask]

                    predictions = self.model(seasons)    
                    # now here, we want to compute the loss between the predicted values
                    # for each season and the actual values for each season
                    # TO-DO: random select grouth truth or predicted value in next timestep
                    loss = self.loss_fn(predictions[mask].squeeze(1), targets) 
                    test_loss.append(loss.item())
            
            train_epoch_loss.append(np.nanmean(train_loss))
            test_epoch_loss.append(np.nanmean(test_loss))
            
            if ep%5 == 1:
                print(f'epoch: {ep:3} last item loss: {loss.item():10.8f}')
                print(f'epoch: {ep:3} train avg. loss: {np.nanmean(train_loss):10.8f}')
                print(f'epoch: {ep:3} test avg. loss: {np.nanmean(test_loss):10.8f}')
                
                
                
            trial.report(np.nanmean(train_loss), ep)

            # Handle pruning based on the intermediate value.
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        print(f'Total Model Training Runtime: {(time.time() - self.start)//60:10.8f} mins')
        self.train_loss = train_epoch_loss
        self.test_loss = test_epoch_loss  
        
        return np.nanmean(self.train_loss)

if __name__ == "__main__":
    
    # Get the player dataset.
    X, y, train_seq, train_target, test_seq, test_target, train_idx, test_idx, train_real_values, test_real_values, train_idx_bool, test_idx_bool = get_player_dataset()

    study = optuna.create_study(direction="minimize")
    study.optimize(train_model, n_trials=5,n_jobs=1)

    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2020-09-22 11:10:52,911] A new study created in memory with name: no-name-45ce6334-076a-44db-8765-4daeac8cb862


Running epoch: 1
epoch:   1 last item loss: 0.30796739
epoch:   1 train avg. loss: 0.29547159
epoch:   1 test avg. loss: 0.28887420
Running epoch: 2
Running epoch: 3
Running epoch: 4
Running epoch: 5
Running epoch: 6
epoch:   6 last item loss: 0.30796739
epoch:   6 train avg. loss: 0.28878262
epoch:   6 test avg. loss: 0.28887420
Running epoch: 7
Running epoch: 8
Running epoch: 9
Total Model Training Runtime: 151.00000000 mins


[I 2020-09-22 13:41:54,811] Trial 0 finished with value: 0.2895258411326466 and parameters: {'lr': 0.04649952469467882}. Best is trial 0 with value: 0.2895258411326466.


Running epoch: 1
epoch:   1 last item loss: 0.30796739
epoch:   1 train avg. loss: 0.28878262
epoch:   1 test avg. loss: 0.28887420
Running epoch: 2
Running epoch: 3
Running epoch: 4
Running epoch: 5
Running epoch: 6
epoch:   6 last item loss: 0.30796739
epoch:   6 train avg. loss: 0.28878262
epoch:   6 test avg. loss: 0.28887420
Running epoch: 7
Running epoch: 8
Running epoch: 9
Total Model Training Runtime: 66.00000000 mins


[I 2020-09-22 14:48:14,619] Trial 1 finished with value: 0.28878262200996874 and parameters: {'lr': 0.0019122238090015832}. Best is trial 1 with value: 0.28878262200996874.


Running epoch: 1
epoch:   1 last item loss: 0.30796739
epoch:   1 train avg. loss: 0.29112288
epoch:   1 test avg. loss: 0.28887420
Running epoch: 2
Running epoch: 3
Running epoch: 4
Running epoch: 5
Running epoch: 6
